# Matrix Factorizations

In [1]:
import numpy as np
import pandas as pd
import spacy
import scipy.sparse
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import NMF
import pymc3 as pm
import theano
import theano.tensor as tt

In [2]:
def mask(token):
    # Helper function to mask out non-tokens
    if (not token.is_ascii
            or token.is_stop
            or token.like_num
            or token.pos_ in ['X', 'SYM']):
        return False
    return True


def tokenize(document):
    # Tokenize by lemmatizing
    doc = nlp(document)
    return [token.lemma_ for token in doc if mask(token)]

In [3]:
# Disable tagger, parser and named-entity recognition
nlp = spacy.load('en', disable=['tagger', 'parser', 'ner'])

# Read data
DATA_FILE = 'NeutralPolitics.csv'
data = pd.read_csv(DATA_FILE).squeeze()

In [4]:
data.head()

0     what points to no collusion which of these cl...
1    the unhrc replaced the un commission on human ...
2    when they replaced the terrible commission on ...
3    the flores decision flores v reno and subseque...
4    the protestors in question are being led bycom...
Name: text, dtype: object

In [5]:
# Vectorize data using tf-idfs
vectorizer = TfidfVectorizer(strip_accents='unicode',
                             tokenizer=tokenize,
                             max_df=0.90,
                             min_df=0.001,
                             norm='l2')

tfidf = vectorizer.fit_transform(data)
feature_names = vectorizer.get_feature_names()

## NMF (Non-Negative Matrix Factorization)

Some people in the field of collaborative filtering refer to this method as SVD, despite it having very little to do with the [SVD from linear algebra](https://en.wikipedia.org/wiki/Singular-value_decomposition).

In [ ]:
# Factorize with NMF.
nmf = NMF(n_components=20,
          random_state=1618,
          alpha=0.2)  # L2 regularization

W = nmf.fit_transform(tfidf)
H = nmf.components_
err = nmf.reconstruction_err_

In [ ]:
# Print clusters and exemplars.
for topic_idx, [scores, topic] in enumerate(zip(np.transpose(W), H)):
    print('Cluster #{}:'.format(topic_idx))
    print('Cluster importance: {}'.format(
        float((np.argmax(W, axis=1) == topic_idx).sum()) / W.shape[0]))

    for token, importance in zip(
            [feature_names[i] for i in np.argsort(topic)[:-10 - 1:-1]],
            np.sort(topic)[:-15 - 1:-1]):
        print('{}: {:2f}'.format(token, importance))

    print('')

    for exemplar_idx in np.argsort(scores)[-5:]:
        print(exemplar_idx)
        print(data[exemplar_idx])
        print('')

    print('----------')

## PMF (Probabilistic Matrix Factorization)

In [6]:
def sparse_std(x, axis=None):
    """ Standard deviation of a scipy.sparse matrix, via [E(X^2) - E(X)^2]^(1/2) """
    return np.sqrt(np.mean(x.power(2), axis=axis) - np.square(np.mean(x, axis=axis)))

In [7]:
rows, columns, entries = scipy.sparse.find(tfidf)

n, m = tfidf.shape
dim = 20

sigma = entries.std()
sigma_u = sparse_std(tfidf, axis=1).mean()
sigma_v = sparse_std(tfidf, axis=0).mean()

In [8]:
'''
# Naive implementation, will not work.
with pm.Model() as pmf:
    U = pm.Normal('U', mu=0, sd=sigma_u, shape=[n, dim])
    V = pm.Normal('V', mu=0, sd=sigma_v, shape=[m, dim])
    R = pm.Normal('R', mu=tt.dot(U, V.T), sd=sigma, shape=[n, m], observed=tfidf)

    map_estimate = pm.find_MAP()
''';

In [9]:
# This doesn't seem to work either? U and V turn out completely 0.
# MAP is an unreliable point...
with pm.Model() as pmf:
    U = pm.Normal('U', mu=0, sd=sigma_u, shape=[n, dim])
    V = pm.Normal('V', mu=0, sd=sigma_v, shape=[m, dim])
    R_nonzero = pm.Normal('R_nonzero',
                          mu=tt.sum(np.multiply(U[rows, :], V[columns, :]), axis=1),
                          sd=sigma,
                          observed=entries)
    
    map_estimate = pm.find_MAP()

In [ ]:
# Sampling is takes a prohibitively long time...
with pmf:
    trace = pm.sample()

## Bayesian Probabilistic Matrix Factorization (BPMF)

In [26]:
N, M = tfidf.shape
D = 20

In [34]:
with pm.Model() as bpmf:
    beta_0 = 2
    mu_0 = np.zeros(shape=D)
    nu_0 = D
    W_0 = np.identity(D)
    
    # Instead of Wishart priors, we use LKJ priors on the correlations, as
    # that is more numerically stable: https://docs.pymc.io/notebooks/LKJ.html
    L_U = pm.LKJCholeskyCov('L_U',
                            n=D,
                            eta=D,
                            sd_dist=pm.HalfNormal.dist(1))
    lambda_U_chol = pm.expand_packed_triangular(D, L_U)

    L_V = pm.LKJCholeskyCov('L_V',
                            n=D,
                            eta=D,
                            sd_dist=pm.HalfNormal.dist(1))    
    lambda_V_chol = pm.expand_packed_triangular(D, L_V)

    mu_U = pm.MvNormal('mu_U',
                       mu=mu_0,
                       chol=np.sqrt(beta_0)*lambda_U_chol,
                       shape=[D,])
    mu_V = pm.MvNormal('mu_V',
                       mu=mu_0,
                       chol=np.sqrt(beta_0)*lambda_V_chol,
                       shape=[D,])
    
    #U = pm.Normal('U', mu=mu_U, sd=lambda_U, shape=[n, dim])
    #V = pm.Normal('V', mu=mu_V, sd=lambda_V, shape=[m, dim])

## References

[1] https://papers.nips.cc/paper/3208-probabilistic-matrix-factorization.pdf

[2] https://www.cs.toronto.edu/~amnih/papers/bpmf.pdf

[3] http://www.cs.toronto.edu/~rsalakhu/BPMF.html (code for [2])

## Appendix

SQL query for data:

```sql
SELECT
  -- In order: lowercase, strip URLs, HTML entities, and punctuation, and replace whitespace with single spaces
  REGEXP_REPLACE(
    REGEXP_REPLACE(
      REGEXP_REPLACE(
        REGEXP_REPLACE(
          LOWER(body),
          -- URL regular expression modified from https://daringfireball.net/2010/07/improved_regex_for_matching_urls
          r"(?i)\b((?:https?://|www\d{0,3}[.]|[a-z0-9.\-]+[.][a-z]{2,4}/)(?:[^\s()<>]+|\(([^\s()<>]+|(\([^\s()<>]+\)))*\))+(?:\(([^\s()<>]+|(\([^\s()<>]+\)))*\)|[^\s[:punct:]]))",
          ""),
        "&([a-z]|#)[a-z]*;",
        ""),
      r"[[:punct:]]",
      ""),
    r"[[:space:]]+",
    " ") AS text
FROM
  `fh-bigquery.reddit_comments.2018_06`
WHERE
  subreddit = 'NeutralPolitics'
ORDER BY
  LENGTH(text) DESC
LIMIT
  10000
```